# Basic Usage

This example on CO illustates all the features of using horton-part package.

The format checkpoint file wavefunction is included in "docs/notebooks/data/co.fchk" and is
read using the [IOData](https://github.com/theochem/iodata) package.

In [1]:
import numpy as np
from iodata import load_one
from gbasis.evals.eval import evaluate_basis
from gbasis.wrappers import from_iodata
from grid import ExpRTransform, UniformInteger, BeckeWeights, MolGrid


def prepare_grid_and_dens():
    """Prepare molecular grid and density."""
    mol = load_one('data/co.fchk')

    # Specify the integration grid
    rtf = ExpRTransform(5e-4, 2e1, 120 - 1)
    uniform_grid = UniformInteger(120)
    rgrid = rtf.transform_1d_grid(uniform_grid)
    becke = BeckeWeights()
    grid = MolGrid.from_preset(
        mol.atnums, mol.atcoords, rgrid, "fine", becke, rotate=False, store=True
    )

    # Get the spin-summed density matrix
    one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
    basis, coord_types = from_iodata(mol)
    basis_grid = evaluate_basis(basis, grid.points, coord_type=coord_types)
    rho = np.einsum("ab,bp,ap->p", one_rdm, basis_grid, basis_grid, optimize=True)
    nelec = grid.integrate(rho)
    print(f"The number of electrons: {nelec}")
    print(f"Coordinates of the atoms \n {mol.atcoords}")
    print(f"Atomic numbers of the atom \n {mol.atnums}")
    return mol, grid, rho


## ISA method

In [2]:
from horton_part import ISAWPart

def main_isa():
    mol, grid, rho = prepare_grid_and_dens()
    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
    }

    part = ISAWPart(**kwargs)
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_isa()


The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing cartesian and pure AIM multipoles and radial AIM moments.
charges:
[ 0.19844372 -0.19856088]
cartesian multipoles:
[[ 1.98443716e-01  1.10081030e-16  1.20583349e-15 -3.03946476e-01
  -3.87895465e+00 -1.36477346e-15  8.31544710e-16 -3.87895465e+00
  -7.78959991e-16 -4.77393559e+00  2.85518570e-15  1.63734207e-15
  -2.35286418e-01  1.53083221e-15 -5.84653386e-16  1.63575629e-15
   1.09603282e-14 -2.35286418e-01  1.72518206e-15 -1.16023931e+00]
 [-1.98560882e-01  1.35583310e-16  7.56012771e-17 -1.28707660e-02
  -3.74640413e+00 -2.00442265e-15  4.07249533e-16 -3.74640413e+00
   1.44724371e-16 -3.83832726e+00 -3.85328999e-16  8.77128408e-16
   1.15252270e-01  1.2521

## MBIS method

In [3]:
from horton_part import MBISWPart

def main_mbis():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
    }

    part = MBISWPart(**kwargs)
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_mbis()

The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1


/Users/yingxing/Projects/horton-part/src/horton_part/utils.py:541: UserWarning: The sum of pro-atom parameters is not equal to atomic population
  warnings.warn(


Computing cartesian and pure AIM multipoles and radial AIM moments.
charges:
[ 0.24139112 -0.24124096]
cartesian multipoles:
[[ 2.41391119e-01  4.90131889e-17  1.35173606e-15 -3.24210581e-01
  -3.82813189e+00 -1.26087591e-15  1.17158854e-15 -3.82813189e+00
  -1.08442440e-15 -4.77618723e+00  1.38081792e-16  2.54710110e-15
  -2.38550535e-01  7.02992077e-16 -9.73044313e-16 -8.00173222e-17
   1.35949794e-14 -2.38550535e-01  3.29132096e-15 -1.01878984e+00]
 [-2.41240964e-01  1.07404043e-16 -3.12158883e-18 -8.43584902e-02
  -3.79693136e+00 -2.12331085e-15  2.24041054e-16 -3.79693136e+00
   3.30073315e-16 -3.94537968e+00  1.94378344e-15  7.05889298e-17
   1.00430597e-02  8.63250701e-16 -1.37292548e-15 -1.22127104e-15
   3.45289467e-15  1.00430597e-02  1.03724149e-15 -1.31736368e-01]]


## GISA method

In [4]:
from horton_part import GaussianISAWPart

def main_gisa():
    mol, grid, rho = prepare_grid_and_dens()
    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GaussianISAWPart(**kwargs)
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_gisa()

The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing cartesian and pure AIM multipoles and radial AIM moments.
charges:
[ 0.15212468 -0.15225026]
cartesian multipoles:
[[ 1.52124677e-01  2.73601984e-16  1.32222322e-15 -2.26375080e-01
  -4.07392305e+00 -1.34573763e-15  3.02970173e-16 -4.07392305e+00
  -6.14690321e-16 -5.04346073e+00  4.21993733e-15  1.31125164e-15
   1.00440469e-01  2.16476488e-15 -8.72436609e-16  3.65015024e-15
   1.07430830e-14  1.00440469e-01  3.50216150e-16 -3.22249742e-01]
 [-1.52250262e-01  1.24585071e-17 -2.22322812e-18  8.58694210e-03
  -3.55113594e+00 -2.02936793e-15  6.37886581e-16 -3.55113594e+00
  -1.90701726e-16 -3.68859780e+00 -1.97009904e-15  1.22841212e-15
   1.96373155e-01 -5.5745

## LISA method

In [5]:
from horton_part import LinearISAWPart

def main_lisa():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = LinearISAWPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa()

The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -6.1341e+00  7e+00  1e+00  1e+00
 1:  1.3115e-02 -1.1385e+00  2e+00  3e-01  4e-01
 2:  5.4347e-02 -4.2828e-01  5e-01  2e-02  1e-02
 3:  6.7496e-03 -6.2831e-02  9e-02  1e-02  7e-03
 4:  3.4753e-03 -1.3899e-02  3e-02  4e-03  3e-03
 5:  4.1595e-03  1.5886e-03  5e-03  8e-04  7e-04
 6:  4.6201e-03  4.2298e-03  7e-04  1e-04  9e-05
 7:  4.5316e-03  4.4480e-03  2e-04  3e-05  2e-05
 8:  4.4898e-03  4.4750e-03  3e-05  5e-06  4e-06
 9:  4.4783e-03  4.4771e-03  2e-06  5e-07  3e-07
10:  4.4771e-03  4.4771e-03  4e-08  9e-09  7e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -7.8428e+00  7e+00  1e+00  1e+00
 1:  8.6157e-02 -1.3020e+00  3e+00  5e-01  5e-01
 2:  1.0075e-01 -6.1060e-01  8e-01  2e-02  8e-03
 3: -1.1627e-02

## Global LISA-101

In [6]:
from horton_part.lisa_g import GlobalLinearISA101WPart

def main_lisa_g_101():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA101WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])
    
main_lisa_g_101()

The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.3888e+01  1e+01  1e+00  1e+00
 1:  1.5544e-01 -2.9659e+00  5e+00  5e-01  5e-01
 2:  2.5074e-01 -1.2986e+00  2e+00  9e-02  9e-02
 3:  9.3163e-02 -3.3206e-01  5e-01  3e-02  2e-02
 4:  8.2543e-02  5.1641e-03  1e-01  8e-03  6e-03
 5:  8.8954e-02  7.6087e-02  2e-02  2e-03  2e-03
 6:  9.2165e-02  9.0777e-02  2e-03  3e-04  2e-04
 7:  9.2405e-02  9.2198e-02  4e-04  5e-05  3e-05
 8:  9.2352e-02  9.2307e-02  8e-05  1e-05  8e-06
 9:  9.2329e-02  9.2322e-02  1e-05  2e-06  1e-06
10:  9.2323e-02  9.2323e-02  1e-06  1e-07  1e-07
11:  9.2323e-02  9.2323e-02  2e-08  2e-09  2e-09
Optimal solution found.
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing cartesian

In [8]:
from horton_part.lisa_g import GlobalLinearISA104WPart

def main_lisa_g_104():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA104WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa_g_104()


The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00  1.1154e-01  1e+00  1e+00  1e+00
 1:  6.8539e-02  9.3075e-02  1e-02  3e-02  2e-02
 2:  9.1305e-02  9.2264e-02  1e-04  1e-03  9e-04
 3:  9.2238e-02  9.2255e-02  1e-06  2e-05  2e-05
 4:  9.2255e-02  9.2255e-02  1e-08  2e-07  2e-07
Optimal solution found.
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing cartesian and pure AIM multipoles and radial AIM moments.
charges:
[ 0.19805266 -0.1982036 ]
cartesian multipoles:
[[ 1.98052662e-01  1.68416749e-16  1.28051185e-15 -2.96693323e-01
  -3.89205580e+00 -1.50446812e-15  5.42176258e-16 -3.89205580e+00
  -6.47461215e-16 -4.80652631e+00  4.26322402e-15  1.12960853e-15
  -1.88298788e-01  2.05777672e-15 -1.93

/Users/yingxing/Projects/horton-part/src/horton_part/utils.py:523: UserWarning: Not all pro-atom parameters are positive!
  warnings.warn("Not all pro-atom parameters are positive!")


In [7]:
from horton_part.lisa_g import GlobalLinearISA301WPart

def main_lisa_g_301():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA301WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa_g_301()


The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |   1   |   0   | +1.1150e-01 | 1.00e+00 | 5.28e-02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   0   |
|   2   |   2   |   2   | +1.5542e-01 | 6.03e+00 | 6.83e-02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   3   |   3   |   4   | +1.6787e-01 | 1.27e+01 | 2.30e-02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   4   |   3   |   4   | +1.6787e-01 | 6.35e+01 | 6.25e-02 | 0.00e+00 | 1.00e+00 |  2.00e-02   |   0   |
|   5   |   4   |   7   | +1.2463e-01 | 6.35e+01 | 3.49e-02 | 0.00e+00 | 1.00e+00 |  2.00e-02   |   4   |
|   6   |   5   |  10   | +1.0803e-01 | 6.35e+01 | 1.48e-02 | 0.00e+00 | 1.00e+

/Users/yingxing/Packages/mambaforge/envs/isa-env/lib/python3.10/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


|  20   |  24   |  60   | +9.7053e-02 | 1.68e-01 | 5.41e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   2   |
|  21   |  25   |  61   | +9.6900e-02 | 1.68e-01 | 5.07e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   4   |
|  22   |  26   |  68   | +9.6251e-02 | 9.84e-01 | 4.48e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   4   |
|  23   |  27   |  72   | +9.5885e-02 | 9.84e-01 | 5.83e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   4   |
|  24   |  29   |  80   | +9.5885e-02 | 1.92e-01 | 5.83e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   4   |
|  25   |  30   |  86   | +9.5104e-02 | 1.12e+00 | 2.79e-03 | 0.00e+00 | 1.00e+00 |  4.00e-03   |   4   |
|  26   |  30   |  86   | +9.5104e-02 | 5.62e+00 | 3.81e-03 | 0.00e+00 | 1.00e+00 |  8.00e-04   |   0   |
|  27   |  31   |  93   | +9.3769e-02 | 5.62e+00 | 2.54e-03 | 0.00e+00 | 1.00e+00 |  8.00e-04   |   4   |
|  28   |  32   |  102  | +9.3259e-02 | 5.62e+00 | 2.10e-03 | 0.00e+00 | 1.00e+00 |  8.00e-04   |   4   |
|  29   |  34   |  108  | +9.3259e-02 | 5.62e-

In [9]:
from horton_part.lisa_g import GlobalLinearISA302WPart

def main_lisa_g_302():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA302WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa_g_302()


The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------|
|   1   |   1   |   0   | +1.1150e-01 | 1.00e+00 | 5.41e-02 | 5.60e-05 | 1.00e+00 |   0   |
|   2   |   2   |   1   | +1.0376e-01 | 1.00e+00 | 3.89e-02 | 0.00e+00 | 1.00e+00 |   4   |
|   3   |   4   |   3   | +1.0376e-01 | 1.73e-01 | 3.89e-02 | 0.00e+00 | 1.00e+00 |   2   |
|   4   |   5   |   5   | +9.5693e-02 | 1.21e+00 | 1.85e-02 | 0.00e+00 | 1.00e+00 |   2   |
|   5   |   6   |   8   | +9.4024e-02 | 1.21e+00 | 1.26e-02 | 1.78e-15 | 1.00e+00 |   4   |
|   6   |   7   |  10   | +9.3873e-02 | 1.21e+00 | 1.11e-02 | 0.00e+00 | 1.00e+00 |   4   |
|   7   |   8   |  14   | +9.3873e-02 | 1.37e-01 | 1.11e-02 | 0.00e+00 | 1.00e+00 |  

In [2]:
from horton_part.lisa_g import GlobalLinearISA201WPart

def main_lisa_g_201():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA201WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa_g_201()



The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]
Computing atomic populations.
Computing atomic charges.
Computing density decomposition for atom 0
Computing density decomposition for atom 1
Computing cartesian and pure AIM multipoles and radial AIM moments.
charges:
[ 0.19868617 -0.19883453]
cartesian multipoles:
[[ 1.98686167e-01  1.67003570e-16  1.21726466e-15 -2.97222544e-01
  -3.89260795e+00 -1.51471977e-15  5.51612747e-16 -3.89260795e+00
  -6.28187046e-16 -4.80702148e+00  4.26529444e-15  1.15399679e-15
  -1.86810377e-01  1.96395376e-15 -1.72195218e-16  3.06901742e-15
   9.33290171e-15 -1.86810377e-01  7.15989598e-16 -1.00138101e+00]
 [-1.98834532e-01 -1.42079735e-17  5.84177761e-17 -2.00832750e-02
  -3.73245439e+00 -1.89510281e-15  6.42252842e-16 -3.73245439e+00
   7.04078051e-18 -3.83488908e+00 -1.96089807e-15  1.46355151e-15
   9.61039435e-02 -4.3979

In [3]:
from horton_part.lisa_g import GlobalLinearISA206WPart

def main_lisa_g_206():
    mol, grid, rho = prepare_grid_and_dens()

    kwargs = {
        "coordinates": mol.atcoords,
        "numbers": mol.atnums,
        "pseudo_numbers": mol.atnums,
        "grid": grid,
        "moldens": rho,
        "lmax": 3,
        "maxiter": 1000,
        "solver": 1,
    }

    part = GlobalLinearISA206WPart(**kwargs)
    # part.do_partitioning()
    part.do_all()

    print("charges:")
    print(part.cache["charges"])
    print("cartesian multipoles:")
    print(part.cache["cartesian_multipoles"])

main_lisa_g_206()




The number of electrons: 13.999981376822717
Coordinates of the atoms 
 [[0.         0.         2.13728025]
 [0.         0.         0.        ]]
Atomic numbers of the atom 
 [6 8]


/Users/yingxing/Projects/horton-part/src/horton_part/lisa_g.py:563: UserWarning: Negative parameters found!
  warnings.warn("Negative parameters found!")


RuntimeError: Error: inner iteration is not converge!